In [127]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
import requests
from bs4 import BeautifulSoup
from datetime import date
import pandas as pd

In [128]:
url = "https://www.linkedin.com/jobs/search/"

In [129]:
def initialize_driver():
    options = webdriver.ChromeOptions()
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    service = Service(executable_path='./chromedriver.exe')
    driver = webdriver.Chrome(service=service, options=options)
    return driver

In [130]:
driver =  initialize_driver()

In [131]:
driver.get(url)

In [133]:
driver.execute_script("window.scrollBy(0, document.body.scrollHeight);")

In [140]:
cnt = 5
while(cnt > 0):
    try:
        more_btn = jobs_div = WebDriverWait(driver,0.5).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR,'button[aria-label="See more jobs"]'))
        )
        more_btn.click()
        cnt -= 1
    except:
        driver.execute_script("window.scrollBy(0, document.body.scrollHeight);")
        

In [141]:
jobs_div = WebDriverWait(driver,5).until(
    EC.element_to_be_clickable((By.CLASS_NAME,"jobs-search__results-list"))
)

In [142]:
jobs = jobs_div.find_elements(By.CSS_SELECTOR,"a[data-tracking-control-name='public_jobs_jserp-result_search-card']")

In [143]:
jobs[0].get_attribute('href')

'https://www.linkedin.com/jobs/view/supply-chain-manager-at-liquid-death-3837322061?refId=KtEhavQySZ4ZZGIocYVqYw%3D%3D&trackingId=cdjHSio%2FuCNDDuzruzhcdw%3D%3D&position=1&pageNum=0&trk=public_jobs_jserp-result_search-card'

In [159]:
def get_job_data(job_link):
    try:
        r = requests.get(job_link)
        soup = BeautifulSoup(r.text, 'html.parser')
        
        job_title = soup.find('h1', {'class': 'topcard__title'}).text.strip()
        company_element = soup.find('a', {'class':'topcard__org-name-link'})
        company_name = company_element.text.strip()
        company_linkedin_url  = company_element.get('href').strip()
        company_location = soup.find('span',{'class':'topcard__flavor topcard__flavor--bullet'}).text.strip()
        job_posted = soup.find('span',{'class':'posted-time-ago__text'}).text.strip()
        today = str(date.today())
        
        job_data = {
            'job_title': job_title,
            'company_name': company_name,
            'company_location': company_location,
            'job_posted': job_posted,
            'scrape_date': today,
            'job_url': job_link,
            'company_linkedin_url': company_linkedin_url,
        }
        return job_data
    except Exception as e:
        print(f"{e} \n issued joblink{job_link}")
        return {}

In [145]:
job_links = []
for job in jobs:
    link = job.get_attribute('href')
    job_links.append(link)

In [146]:
len(job_links)

271

In [150]:
job_data = []
for link in job_links:
    data = get_job_data(link)
    job_data.append(data)

https://www.linkedin.com/jobs/view/help-desk-support-specialist-i-at-soc-llc-3837319442?refId=KtEhavQySZ4ZZGIocYVqYw%3D%3D&trackingId=1t4UHsWs2lkd4oiNQpeC%2Fg%3D%3D&position=24&pageNum=0&trk=public_jobs_jserp-result_search-card
https://www.linkedin.com/jobs/view/recruiter-at-ascendion-3838739674?refId=9gf8Iveg%2B8bFKPiCYtVWjg%3D%3D&trackingId=zFRG3ncJTBF9pjpUvhwPgw%3D%3D&position=4&pageNum=1&trk=public_jobs_jserp-result_search-card
https://www.linkedin.com/jobs/view/human-resources-business-partner-at-trulieve-3837314411?refId=9gf8Iveg%2B8bFKPiCYtVWjg%3D%3D&trackingId=LWB2%2FMOS7vLpbId%2FUmFfMA%3D%3D&position=9&pageNum=1&trk=public_jobs_jserp-result_search-card
https://www.linkedin.com/jobs/view/administrative-coordinator-at-wedi-3838755131?refId=9gf8Iveg%2B8bFKPiCYtVWjg%3D%3D&trackingId=vkcpNvFm639N4zV1vsM8yw%3D%3D&position=11&pageNum=1&trk=public_jobs_jserp-result_search-card
https://www.linkedin.com/jobs/view/scheduling-coordinator-remote-at-feyerassociates-inc-3837088427?refId=nSkB

In [158]:
df = pd.DataFrame.from_dict(job_data)
df = df.dropna()
df.to_csv("Linkedin_Jobs.csv", index=False)